In [2]:
import argparse
from argparse import ArgumentParser


def get_argsparser() -> ArgumentParser:
    parser: ArgumentParser = argparse.ArgumentParser(description="这是一个示例")
    # 添加参数
    parser.add_argument('positional_arg', type=int, help='一个位置参数')
    parser.add_argument('--flag', action='store_true', help='一个标记参数')
    parser.add_argument('-p', '--param', type=str, default='default_value', help='一个可选参数')
    
    parser.add_argument(
        "--model_dir",
        type=str,
        default=None,
        help=("Directory include:'model.pdiparams', 'model.pdmodel', "
              "'infer_cfg.yml', created by tools/export_model.py."),
        required=False)
    parser.add_argument(
        "--image_file", type=str, default=None, help="Path of image file.")
    parser.add_argument(
        "--image_dir",
        type=str,
        default=None,
        help="Dir of image file, `image_file` has a higher priority.")
    parser.add_argument(
        "--batch_size", type=int, default=1, help="batch_size for inference.")
    parser.add_argument(
        "--video_file",
        type=str,
        default=None,
        help="Path of video file, `video_file` or `camera_id` has a highest priority."
    )
    parser.add_argument(
        "--camera_id",
        type=int,
        default=-1,
        help="device id of camera to predict.")
    parser.add_argument(
        "--threshold", type=float, default=0.5, help="Threshold of score.")
    parser.add_argument(
        "--output_dir",
        type=str,
        default="output",
        help="Directory of output visualization files.")
    parser.add_argument(
        "--run_mode",
        type=str,
        default='fluid',
        help="mode of running(fluid/trt_fp32/trt_fp16/trt_int8)")
    parser.add_argument(
        "--device",
        type=str,
        default='cpu',
        help="Choose the device you want to run, it can be: CPU/GPU/XPU, default is CPU."
    )
    parser.add_argument(
        "--use_gpu",
        type=ast.literal_eval,
        default=False,
        help="Deprecated, please use `--device`.")
    parser.add_argument(
        "--run_benchmark",
        type=ast.literal_eval,
        default=False,
        help="Whether to predict a image_file repeatedly for benchmark")
    parser.add_argument(
        "--enable_mkldnn",
        type=ast.literal_eval,
        default=False,
        help="Whether use mkldnn with CPU.")
    parser.add_argument(
        "--cpu_threads", type=int, default=1, help="Num of threads with CPU.")
    parser.add_argument(
        "--trt_min_shape", type=int, default=1, help="min_shape for TensorRT.")
    parser.add_argument(
        "--trt_max_shape",
        type=int,
        default=1280,
        help="max_shape for TensorRT.")
    parser.add_argument(
        "--trt_opt_shape",
        type=int,
        default=640,
        help="opt_shape for TensorRT.")
    parser.add_argument(
        "--trt_calib_mode",
        type=bool,
        default=False,
        help="If the model is produced by TRT offline quantitative "
             "calibration, trt_calib_mode need to set True.")
    parser.add_argument(
        '--save_images',
        action='store_true',
        help='Save visualization image results.')
    parser.add_argument(
        '--save_mot_txts',
        action='store_true',
        help='Save tracking results (txt).')
    parser.add_argument(
        "--reid_model_dir",
        type=str,
        default=None,
        help=("Directory include:'model.pdiparams', 'model.pdmodel', "
              "'infer_cfg.yml', created by tools/export_model.py."))
    parser.add_argument(
        "--reid_batch_size",
        type=int,
        default=50,
        help="max batch_size for reid model inference.")
    parser.add_argument(
        '--use_dark',
        type=bool,
        default=True,
        help='whether to use darkpose to get better keypoint position predict ')
    return parser


import argparse

def main():
    # 创建 ArgumentParser 对象
    parser = get_argsparser()

    # 解析参数
    args = parser.parse_args()

    # 使用参数
    if args.flag:
        print('标记参数已启用')
    print('位置参数的值：', args.positional_arg)
    print('可选参数的值：', args.param)

if __name__ == '__main__':
    main()


In [5]:
import requests
from typing import Tuple
import numpy as np
from scipy.io import wavfile
from scipy.io.wavfile import read, write


class DamoTTS():
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    def __init__(self) -> None:
        super().__init__()
        self.url = "http://59.68.29.90:17902/tts_damo"
    
    def tts_run(self, text, out_path, spk_id="zhitian_emo") -> Tuple[np.ndarray, str]:
        payload = {
            "text": text,
            "model": "sambert-hifigan",
            "spk_id": spk_id
        }
        response = requests.post(self.url, data=payload, headers=self.headers)
        open(out_path, 'wb').write(response.content)
        output = wavfile.read(out_path) # output[0], the sampling rate, output[1], the sample array you want to analyze
        
        return output[1], out_path # type: ignore
    
damo = DamoTTS()

damo.tts_run("你好", "1.wav")

(array([-478, -479, -497, ...,  -46,  -35,  -25], dtype=int16), '1.wav')